In [25]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  1 if c['tag'] == 'BTC' else c['exchanges'][0]['price'], c['algorithm'], c['nethash'] ])
df = pd.DataFrame(data, index=index, columns=columns)
df['block_reward_btc'] = df.block_reward * df.exchange_rate_btc
df['daily_emmisions_btc'] = 86400 / df.block_time * df.block_reward_btc
df

,block_time,block_reward,exchange_rate_btc,algorithm,nethash,block_reward_btc,daily_emmisions_btc
BCH,609.0000,3.125000,3.410623e-03,SHA-256,2649132618349513957,0.010658,1.512099
BTC,663.0000,3.158051,1.000000e+00,SHA-256,700315690625637858220,3.158051,411.546896
DOGE,63.0000,10000.000000,2.709000e-06,Scrypt,2909032600071622,0.027090,37.152000
KAS,0.9995,65.421518,9.975386e-07,kHeavyHash,1385449409073613806,0.000065,5.641327
LTC,150.0000,6.250000,1.071600e-03,Scrypt,2129993045490122,0.006698,3.857760
RVN,60.0000,2500.000000,1.534879e-07,KawPow,6833541749549,0.000384,0.552556


In [ ]:
import requests, json
import pandas as pd

algorithms = { 'SCRYPT', 'SHA256ASICBOOST', 'KHEAVYHASH', 'KAWPOW'}

multialgo_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = multialgo_response.json()
algo_data = { element['algorithm']: {'paying': element['paying'], 'speed': element['speed']} for element in multialgo_data['miningAlgorithms'] if element['algorithm'] in algorithms }

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
algorithms_response.raise_for_status()
algorithms_data = algorithms_response.json()

index = [ element['algorithm'] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]
columns = ['marketFactor', 'displayMarketFactor', 'priceFactor', 'displayPriceFactor', 'miningFactor', 'displayMiningFactor', 'priceDownStep', 'paying', 'speed']
data = [ [ float(element['marketFactor']), element['displayMarketFactor'], float(element['priceFactor']), element['displayPriceFactor'], float(element['miningFactor']), element['displayMiningFactor'], float(element['priceDownStep']), algo_data[element['algorithm']]['paying'], algo_data[element['algorithm']]['speed'] ] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]

raw_data_df = pd.DataFrame(data, index=index, columns=columns)

calc_df = pd.DataFrame(index=index)
sats2btc = 1e8

calc_df['hashrate'] = raw_data_df['speed'] / raw_data_df['marketFactor'] * raw_data_df['miningFactor']
calc_df['hashrate unit'] = raw_data_df['displayMarketFactor'] + '/s'

calc_df['paying'] = raw_data_df['paying'] * raw_data_df['priceFactor'] / sats2btc
calc_df['paying unit'] = 'BTC/' + raw_data_df['displayPriceFactor'] + '/day'

calc_df['daily_consumption_capacity'] = calc_df['hashrate'] * calc_df['paying']
calc_df['daily_consumption_capacity unit'] = 'BTC/day'

print('Nicehash data:\n')
json.loads(calc_df.to_json(orient='index'))
